In [1]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing

## Загрузка DataFrame

### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad средствами библиотеки Pandas сформируйте поле выводов по каждому анализирумомому параметру. Например, по показателю pH получен результат 8.4 единицы pH при нормативе от 6 до 9 единиц pH. Т.о. по данному показателю результат анализа в норме. Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, 6 >= pH >= 9. В итоговом DataFrame столбец "Показатель" сделайте индексным.

Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).

In [2]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')))
### Если не работает загрузка on-line
# df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,<h1>Forbidden</h1><pre>Request ID: 2022-09-17-10-52-16-530C8272204C8C2D</pre>


In [3]:
df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [4]:
# создаем столбцы с нормативными показателями 
# df['Норматив2'] = df[df['Норматив'].str.contains('не более')]['Норматив'].str.extract(r'(\d+[,\d]*)')
# df['Норматив3'] = df[df['Норматив'].str.contains('в пределах ')]['Норматив'].str.extract(r'(\d+[-\d]*)')
df['Норматив2'] = df['Норматив'].apply(lambda x: x.split()[2])
df['Норматив3'] = df[df['Норматив2'].str.contains('-')]['Норматив2'].apply( lambda x: x.split('-')[0])
df['Норматив4'] = df[df['Норматив2'].str.contains('-')]['Норматив2'].apply( lambda x: x.split('-')[1])


In [5]:
df

,Показатель,Единица измерений,Результат анализа,Норматив,Норматив2,Норматив3,Норматив4
0,pH,единицы pH,8.4,в пределах 6-9,6-9,6,9
1,Запах,баллы,1,не более 2-3,2-3,2,3
2,Цветность,градусы,б/цвета,не более 30,30,NaN,NaN
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,7-10,7,10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5","1,5",NaN,NaN
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3","3,3",NaN,NaN
6,Нитраты (по NO3),мг/дм3,24,не более 45,45,NaN,NaN
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5","3,5",NaN,NaN
8,Хлориды (Cl),мг/дм3,200,не более 350,350,NaN,NaN
9,Сульфаты (SO4),мг/дм3,189.5,не более 500,500,NaN,NaN


In [6]:
# приводим типы данных к числовым
df['Норматив2'] = df['Норматив2'].str.replace(',','.')
# df[~df['Результат анализа'].str.contains('/')].astype({'Результат анализа': np.float})
# df[~df['Норматив2'].str.contains('-')].astype({'Норматив2': np.float})
# df.astype({'Норматив3': np.float, 'Норматив4': np.float})
# df.info()

In [7]:
df1 = df[(df['Норматив'].str.contains('более'))& (~df['Показатель'].isin(['Запах','Цветность']))]
df1 = df1.astype({'Результат анализа' : np.float,
                 'Норматив2': np.float})
df1.loc[df1['Результат анализа'] <= df1['Норматив2'], 'Результат'] = 'good'
df2 = df[df['Норматив'].str.contains('пределах')]
df2 = df2.astype({'Результат анализа' : np.float,
                 'Норматив3': np.float,
                 'Норматив4': np.float})
df2.loc[(df2['Норматив3'] <= df2['Результат анализа']) & (df2['Результат анализа'] <= df2['Норматив4']), 'Результат'] = 'good'
df3 = df[df['Норматив'].str.contains('более')& df['Норматив'].str.contains('-')]
df3 = df3.astype({'Результат анализа' : np.float,
                 'Норматив3': np.float,
                 'Норматив4': np.float})
df3.loc[(df3['Результат анализа'] <= df3['Норматив3']) & (df3['Результат анализа'] <= df3['Норматив4']), 'Результат'] = 'good'
df4 = df[df['Результат анализа'].str.contains('/')]
df = pd.concat([df1, df2, df3, df4])


/var/folders/04/p9n_pr6j0n971vm_1r43l7400000gn/T/ipykernel_95117/285785873.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df1 = df1.astype({'Результат анализа' : np.float,
/var/folders/04/p9n_pr6j0n971vm_1r43l7400000gn/T/ipykernel_95117/285785873.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  'Норматив2': np.float})
/var/folders

In [8]:
df.loc[df['Результат'] != 'good', 'Результат'] = 'BAD'

In [9]:
df_new = df.reindex(columns=['Показатель', 'Единица измерений', 'Результат анализа', 'Норматив', 'Результат']).set_index('Показатель')

df_new

,Единица измерений,Результат анализа,Норматив,Результат
Показатель,,,,
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",good
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",good
Нитраты (по NO3),мг/дм3,24.0,не более 45,good
Фосфаты (P),мг/дм3,0.36,"не более 3,5",good
Хлориды (Cl),мг/дм3,200.0,не более 350,good
Сульфаты (SO4),мг/дм3,189.5,не более 500,good
Железо (включая хлорное железо) по Fe,мг/дм3,0.019,"не более 0,3",good
Нефть,мг/дм3,0.55,"не более 0,3",BAD
Общая минерализация (сухой остаток),мг/дм3,590.0,не более 1000,good


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?

In [10]:
N = 1000 
s = pd.Series(['apple', 'orange'])
p = 0.5
pop = np.random.choice(s, N, p=[p, p])

In [11]:
test = list(map(lambda x: 
                np.random.choice(s, 1, p=[1-p, p])[0]
                if x == s[1] 
                else np.random.choice(s, 1, p=[1-p, p])[0],
                   pop))
df = pd.DataFrame(zip(pop, test), columns = ['fruit1', 'fruit2'])
df['fruit3'] = test
print ('Вероятность, что все три фрукта – апельсины:', df[
      (
          df['fruit1'] == s[0]) 
          & (df['fruit2'] == s[0]) 
          & (df['fruit3'] == s[0])
         ].shape[0]
         /len(df['fruit3'])
      )

Вероятность, что все три фрукта – апельсины: 0.262


В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?
- не соглашусь, вероятность выбора 3х апельсинов выше

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?

В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [12]:
s = pd.Series(['well', 'defect'])
p = 0.5
pop = np.random.choice(s, N, p=[p, p])

In [13]:
test = list(map(lambda x: 
                np.random.choice(s, 1, p=[1-p, p])[0]
                if x == s[1] 
                else np.random.choice(s, 1, p=[1-p, p])[0],
                   pop))
df = pd.DataFrame(zip(pop, test), columns = ['fruit1', 'fruit2'])
# df['fruit3'] = test
print ('Вероятность, что Мастер проверит ровно две детали:', df[
      (
          df['fruit1'] == s[0]) 
          & (df['fruit2'] == s[0])
         ].shape[0]
         /len(df['fruit2'])
      )

Вероятность, что Мастер проверит ровно две детали: 0.244


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?
- подтверждается